In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sqlalchemy import create_engine
from scipy.stats.mstats import winsorize
import seaborn as sns
import warnings

warnings.filterwarnings('ignore')

In [2]:
postgres_user = 'dsbc_student'
postgres_pw = '7*.8G9QH21'
postgres_host = '142.93.121.174'
postgres_port = '5432'
postgres_db = 'useducation'

db_url = f'postgresql://{postgres_user}:{postgres_pw}@{postgres_host}:{postgres_port}/{postgres_db}'

engine = create_engine(db_url)

dsbc_df = pd.read_sql_query('SELECT * FROM useducation;', con=engine)

# close the db connetion
engine.dispose()

In [3]:
df2 = dsbc_df.copy()

In [4]:
df2.head()

,PRIMARY_KEY,STATE,YEAR,ENROLL,TOTAL_REVENUE,FEDERAL_REVENUE,STATE_REVENUE,LOCAL_REVENUE,TOTAL_EXPENDITURE,INSTRUCTION_EXPENDITURE,...,GRADES_4_G,GRADES_8_G,GRADES_12_G,GRADES_1_8_G,GRADES_9_12_G,GRADES_ALL_G,AVG_MATH_4_SCORE,AVG_MATH_8_SCORE,AVG_READING_4_SCORE,AVG_READING_8_SCORE
0,1992_ALABAMA,ALABAMA,1992,NaN,2678885.0,304177.0,1659028.0,715680.0,2653798.0,1481703.0,...,57948.0,58025.0,41167.0,471564.0,196386.0,676174.0,208.327876,252.187522,207.963517,NaN
1,1992_ALASKA,ALASKA,1992,NaN,1049591.0,106780.0,720711.0,222100.0,972488.0,498362.0,...,9748.0,8789.0,6714.0,79117.0,30847.0,112335.0,NaN,NaN,NaN,258.859712
2,1992_ARIZONA,ARIZONA,1992,NaN,3258079.0,297888.0,1369815.0,1590376.0,3401580.0,1435908.0,...,55433.0,49081.0,37410.0,437127.0,175210.0,614881.0,215.253932,265.366278,206.212716,262.169895
3,1992_ARKANSAS,ARKANSAS,1992,NaN,1711959.0,178571.0,958785.0,574603.0,1743022.0,964323.0,...,34632.0,36011.0,27651.0,281338.0,123113.0,405259.0,210.206028,256.312090,208.634458,264.619665
4,1992_CALIFORNIA,CALIFORNIA,1992,NaN,26260025.0,2072470.0,16546514.0,7641041.0,27138832.0,14358922.0,...,418418.0,363296.0,270675.0,3286034.0,1372011.0,4717112.0,208.398961,260.892247,196.764414,NaN


In [5]:
fill_columns = df2.drop(
    ['PRIMARY_KEY', 'STATE','YEAR','AVG_MATH_4_SCORE', 
     'AVG_MATH_8_SCORE','AVG_READING_4_SCORE', 'AVG_READING_8_SCORE'], axis=1).columns

In [6]:
len(fill_columns)

18

In [7]:
states = df2.STATE.unique()
df2.isnull().sum()/df2.isnull().count()

PRIMARY_KEY                     0.000000
STATE                           0.000000
YEAR                            0.000000
ENROLL                          0.176273
TOTAL_REVENUE                   0.142091
FEDERAL_REVENUE                 0.142091
STATE_REVENUE                   0.142091
LOCAL_REVENUE                   0.142091
TOTAL_EXPENDITURE               0.142091
INSTRUCTION_EXPENDITURE         0.142091
SUPPORT_SERVICES_EXPENDITURE    0.142091
OTHER_EXPENDITURE               0.176273
CAPITAL_OUTLAY_EXPENDITURE      0.142091
GRADES_PK_G                     0.115952
GRADES_KG_G                     0.088472
GRADES_4_G                      0.087802
GRADES_8_G                      0.087802
GRADES_12_G                     0.087802
GRADES_1_8_G                    0.087802
GRADES_9_12_G                   0.087802
GRADES_ALL_G                    0.115952
AVG_MATH_4_SCORE                0.640751
AVG_MATH_8_SCORE                0.643432
AVG_READING_4_SCORE             0.642761
AVG_READING_8_SC

In [8]:
for state in states:
    df2.loc[df2['STATE'] == state, fill_columns].interpolate(inplace=True)

In [9]:

df2.dropna(inplace=True)
df2.isnull().sum()/df2.isnull().count()

PRIMARY_KEY                     0.0
STATE                           0.0
YEAR                            0.0
ENROLL                          0.0
TOTAL_REVENUE                   0.0
FEDERAL_REVENUE                 0.0
STATE_REVENUE                   0.0
LOCAL_REVENUE                   0.0
TOTAL_EXPENDITURE               0.0
INSTRUCTION_EXPENDITURE         0.0
SUPPORT_SERVICES_EXPENDITURE    0.0
OTHER_EXPENDITURE               0.0
CAPITAL_OUTLAY_EXPENDITURE      0.0
GRADES_PK_G                     0.0
GRADES_KG_G                     0.0
GRADES_4_G                      0.0
GRADES_8_G                      0.0
GRADES_12_G                     0.0
GRADES_1_8_G                    0.0
GRADES_9_12_G                   0.0
GRADES_ALL_G                    0.0
AVG_MATH_4_SCORE                0.0
AVG_MATH_8_SCORE                0.0
AVG_READING_4_SCORE             0.0
AVG_READING_8_SCORE             0.0
dtype: float64

### 1. Compute the average score for each row for all students weighted properly by grade. In other words, each row will have its own average. Notice that the number of students in the 4th grade isn't the same as the number of students in the 8th grade. So, you should appropriately weigh the scores.


In [10]:
df2.head()

,PRIMARY_KEY,STATE,YEAR,ENROLL,TOTAL_REVENUE,FEDERAL_REVENUE,STATE_REVENUE,LOCAL_REVENUE,TOTAL_EXPENDITURE,INSTRUCTION_EXPENDITURE,...,GRADES_4_G,GRADES_8_G,GRADES_12_G,GRADES_1_8_G,GRADES_9_12_G,GRADES_ALL_G,AVG_MATH_4_SCORE,AVG_MATH_8_SCORE,AVG_READING_4_SCORE,AVG_READING_8_SCORE
209,1996_ALABAMA,ALABAMA,1996,735912.0,3365113.0,334858.0,2138717.0,891538.0,3572283.0,1987018.0,...,57098.0,58305.0,42510.0,469370.0,207980.0,686983.0,211.646974,256.594863,210.920767,260.276441
211,1996_ARIZONA,ARIZONA,1996,764681.0,4143304.0,378917.0,1798363.0,1966024.0,4391555.0,1860007.0,...,62793.0,60514.0,42041.0,511925.0,210289.0,726709.0,217.575940,267.874834,206.529788,263.283638
212,1996_ARKANSAS,ARKANSAS,1996,452907.0,2183384.0,192152.0,1296247.0,694985.0,2297381.0,1264275.0,...,33855.0,37064.0,27613.0,283883.0,131997.0,417860.0,215.846436,261.652745,208.837716,271.834590
213,1996_CALIFORNIA,CALIFORNIA,1996,5460484.0,31282981.0,2603882.0,17061474.0,11617625.0,31561692.0,16880730.0,...,435217.0,406456.0,298669.0,3516572.0,1525618.0,5115423.0,209.129083,262.772478,201.912591,269.197451
214,1996_COLORADO,COLORADO,1996,655679.0,3891203.0,194998.0,1664676.0,2031529.0,4075165.0,2052264.0,...,52524.0,52269.0,37179.0,423510.0,185700.0,621730.0,225.805858,275.608214,221.653243,268.168154


In [11]:
df2['grade_4_weight'] = (df2.GRADES_4_G/(df2.GRADES_4_G + df2.GRADES_8_G))
df2['grade_8_weight'] = (df2.GRADES_8_G/(df2.GRADES_4_G + df2.GRADES_8_G))
df2['average_score'] = ((df2.grade_4_weight*(df2.AVG_READING_4_SCORE + df2.AVG_MATH_4_SCORE)) + 
                     (df2.grade_8_weight*(df2.AVG_READING_8_SCORE + df2.AVG_MATH_8_SCORE)))/2
df2.head()

,PRIMARY_KEY,STATE,YEAR,ENROLL,TOTAL_REVENUE,FEDERAL_REVENUE,STATE_REVENUE,LOCAL_REVENUE,TOTAL_EXPENDITURE,INSTRUCTION_EXPENDITURE,...,GRADES_1_8_G,GRADES_9_12_G,GRADES_ALL_G,AVG_MATH_4_SCORE,AVG_MATH_8_SCORE,AVG_READING_4_SCORE,AVG_READING_8_SCORE,grade_4_weight,grade_8_weight,average_score
209,1996_ALABAMA,ALABAMA,1996,735912.0,3365113.0,334858.0,2138717.0,891538.0,3572283.0,1987018.0,...,469370.0,207980.0,686983.0,211.646974,256.594863,210.920767,260.276441,0.494770,0.505230,235.106342
211,1996_ARIZONA,ARIZONA,1996,764681.0,4143304.0,378917.0,1798363.0,1966024.0,4391555.0,1860007.0,...,511925.0,210289.0,726709.0,217.575940,267.874834,206.529788,263.283638,0.509241,0.490759,238.321404
212,1996_ARKANSAS,ARKANSAS,1996,452907.0,2183384.0,192152.0,1296247.0,694985.0,2297381.0,1264275.0,...,283883.0,131997.0,417860.0,215.846436,261.652745,208.837716,271.834590,0.477376,0.522624,240.773675
213,1996_CALIFORNIA,CALIFORNIA,1996,5460484.0,31282981.0,2603882.0,17061474.0,11617625.0,31561692.0,16880730.0,...,3516572.0,1525618.0,5115423.0,209.129083,262.772478,201.912591,269.197451,0.517086,0.482914,234.719834
214,1996_COLORADO,COLORADO,1996,655679.0,3891203.0,194998.0,1664676.0,2031529.0,4075165.0,2052264.0,...,423510.0,185700.0,621730.0,225.805858,275.608214,221.653243,268.168154,0.501217,0.498783,247.750273


### 2. What are the correlations between this newly created score variable and the expenditure types? Which 1 of the expenditure types is more correlated than the others?

In [12]:
# INSTRUCTION_EXPENDITURE is the most correlated with average score
df2.filter(like='_EXPENDITURE').corrwith(df2.average_score)

TOTAL_EXPENDITURE               0.086797
INSTRUCTION_EXPENDITURE         0.100311
SUPPORT_SERVICES_EXPENDITURE    0.092225
OTHER_EXPENDITURE               0.015186
CAPITAL_OUTLAY_EXPENDITURE      0.001657
dtype: float64

### 3. Now, apply PCA to the 4 expenditure types. How much of the total variance is explained by the 1st component?

In [24]:
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X = df2.filter(like='_EXPENDITURE')
X = scaler.fit_transform(X)
pca = PCA(n_components=1)
df2['pc0'] = pca.fit_transform(X)
pca.explained_variance_ratio_
## 95% of total variance is explained by the 1st component

array([0.95199991])

### 4. What is the correlation between the overall score variable and the 1st principal component?

In [25]:
df2.corr().pc0

YEAR                            0.112623
ENROLL                          0.951556
TOTAL_REVENUE                   0.993307
FEDERAL_REVENUE                 0.940067
STATE_REVENUE                   0.974023
LOCAL_REVENUE                   0.944794
TOTAL_EXPENDITURE               0.995065
INSTRUCTION_EXPENDITURE         0.973935
SUPPORT_SERVICES_EXPENDITURE    0.990751
OTHER_EXPENDITURE               0.965240
CAPITAL_OUTLAY_EXPENDITURE      0.952900
GRADES_PK_G                     0.744475
GRADES_KG_G                     0.938679
GRADES_4_G                      0.942464
GRADES_8_G                      0.949424
GRADES_12_G                     0.964027
GRADES_1_8_G                    0.943418
GRADES_9_12_G                   0.960087
GRADES_ALL_G                    0.949656
AVG_MATH_4_SCORE                0.097019
AVG_MATH_8_SCORE                0.049576
AVG_READING_4_SCORE             0.024078
AVG_READING_8_SCORE             0.018478
grade_4_weight                 -0.012334
grade_8_weight  

### 5. If you were to choose the best variables for your model, would you prefer using the 1st principal component instead of the expenditure variables? Why?

In [22]:
# INSTRUCTION_EXPENDITURE still has a higher correlation with overall score